In [3]:
import torch
import torch.nn as nn
from networks import DynamicUNet, BaseUNet

network = DynamicUNet(1, 5, [1,2,3,4,5])

val = network(torch.randn((5, 1, 64, 64)))
val.size()

torch.Size([5, 5, 64, 64])

In [2]:
from torchviz import make_dot

make_dot(val.sum(), params=dict(list(network.named_parameters()))).render("torchviz3", format="png")

'torchviz3.png'